In [2]:
import pandas as pd
import re

In [3]:
df = pd.read_excel(r'd:\Onedrive\OneDrive - UNI CONSULTING\Attachments\Code\uni_craw_tvpl\uni_craw_section_chapter_header\Craw_ko_en\ko_en_section_chapter_heading_des_notes.xlsx')

In [4]:
def create_chapter_df(df):
    # Select columns: 'enchapter', 'enchapter_notes', 'kochapter_notes'
    df = df.loc[:, ['enchapter', 'enchapter_notes', 'kochapter_notes']]
    # Drop duplicate rows across all columns
    df = df.drop_duplicates()
    return df

chapter_df = create_chapter_df(df.copy())

In [5]:
# for x in chapter_df.loc[:,'enchapter_notes']:
#     print(x)

In [6]:
from bs4 import BeautifulSoup

def fix_atag(html):
    # Phân tích cú pháp HTML
    soup = BeautifulSoup(html, 'html.parser')

    # Duyệt qua tất cả thẻ <a>
    for a_tag in soup.find_all('a'):
        # Kiểm tra nếu thẻ <a> chứa <img>
        if a_tag.find('img'):
            del a_tag['rel']
        else:
            # Thay thẻ <a> bằng text bên trong nó
            a_tag.replace_with(a_tag.text)

    # Kết quả sau khi xử lý
    result = str(soup).replace('"/clip/','"https://unipass.customs.go.kr/clip/')
    print(result)
    return result


In [7]:
def process_chapter_notes(df,en_col,ko_col):
    
    def split_text(text):
        if not isinstance(text, str):
            return []
        text = text.replace('\n \n', '\n\n')
        text = text.replace('_x000D__x000D_', '\n')
        # Thay thế 3 hoặc nhiều '\n' thành '\n\n'
        text = re.sub(r'\n{2,}', '\n', text)
        delimiter = '\n'
        parts = text.split(delimiter)
        # print(parts)
        return parts
    
    def get_note_index(parts):
        note_pattern_en = r'^Notes?\.$'
        note_pattern_ko = r'^주:\s*'
        # Tìm các vị trí phần tử khớp với regex
        note_index = next((i for i, item in enumerate(parts) if re.fullmatch(note_pattern_en, item.replace('\n','').strip()) or re.fullmatch(note_pattern_ko, item.replace('\n','').strip())), None)
        # print("note_index: ",note_index)
        return note_index
    
    def get_general_index(parts):
        general_pattern_en = r'^GENERAL\.*$'
        general_pattern_ko = r'^총설\s*$'
        general_index = next((i for i, item in enumerate(parts) if re.fullmatch(general_pattern_en, item.replace('\n','').strip())or re.fullmatch(general_pattern_ko, item.replace('\n','').strip())), None)
        # print('general_index: ',general_index)
        return general_index
    
    def process_notes(x):
        split = split_text(x)
        note_idx = get_note_index(split)
        general_idx = get_general_index(split)
        if note_idx is None:
            return []
        elif general_idx is not None:
            return split[note_idx:general_idx]
        else:
            return split[note_idx:]
    
    def process_general(x):
        split = split_text(x)
        general_idx = get_general_index(split)
        if general_idx is None:
            return []
        elif general_idx is not None:
            return split[general_idx:]
        
    def process_des(x):
        split = split_text(x)
        note_idx = get_note_index(split)
        if note_idx is None:
            return split[1:]
        elif note_idx is not None:
            return split[1:note_idx]
    # Áp dụng hàm
    df['section_name'] = df[en_col].apply(lambda x: split_text(x)[0])
    df['en_description'] = df[en_col].apply(process_des)
    df['ko_description'] = df[ko_col].apply(process_des)
    
    df['en_notes'] = df[en_col].apply(process_notes)
    
    df['ko_notes'] = df[ko_col].apply(process_notes)
    
    df['en_general'] = df[en_col].apply(process_general)
    df['ko_general'] = df[ko_col].apply(process_general)

    return df


In [8]:
# Gọi hàm xử lý
chapter_df = process_chapter_notes(chapter_df, 'enchapter_notes', 'kochapter_notes')

In [9]:
chapter_df = chapter_df.drop(columns=['enchapter_notes', 'kochapter_notes'])

In [10]:
chapter_df['ko_notes'] = chapter_df['ko_notes'].apply(lambda x: [fix_atag(html) for html in x])
chapter_df['ko_general'] = chapter_df['ko_general'].apply(lambda x: [fix_atag(html) for html in x])

주: 
1. 이 류에는 다음 각 목의 것을 제외한 모든 살아 있는 동물이 포함된다.
가. 제0301호ㆍ제0306호ㆍ제0307호ㆍ제0308호의 어류ㆍ갑각류ㆍ연체동물과 그 밖의 수생(水生) 무척추동물 ;
나. 제3002호의 미생물 배양체와 그 밖의 물품
다. 제9508호의 동물
주: 
1. 이 류에서 다음 각 목의 것은 제외한다.
가. 제0201호부터 제0208호까지 또는 제0210호에서 열거한 물품 중 식용에 적합하지 않은 것
나. 식용에 적합한 죽은 곤충(제0410호)
다. 동물의 장ㆍ방광ㆍ위(제0504호), 동물의 피(제0511호나 제3002호)
라. 제0209호의 물품 외의 동물성 지방(제15류)
주: 
1. 이 류에서 다음 각 목의 것은 제외한다.
가. 제0106호의 포유동물
나. 제0106호의 포유동물의 육(제0208호나 제0210호)
다. 죽은 것으로서 그 종(種)이나 상태로 보아 식용에 적합하지 않은 어류[간, 어란(魚卵)과 어백(魚白)을 포함한다]ㆍ갑각류ㆍ연체동물이나 그 밖의 수생(水生) 무척추동물(제5류), 식용에 적합하지 않은 어류ㆍ갑각류ㆍ연체동물이나 그 밖의 수생(水生) 무척추동물의 고운 가루ㆍ거친 가루나 펠릿(pellet)(제2301호)
라. 캐비어, 어란(魚卵)으로 조제한 캐비아 대용물(제1604호)
2. 이 류에서 “펠릿(pellet)”이란 직접 압축하거나 소량의 점결제를 첨가하여 응결시킨 물품을 말한다.
3. 제0305호부터 제0308호까지에는 식용에 적합한 고운 가루, 거친 가루와 펠릿(pellet)은 포함하지 않는다(제0309호).
주: 
1. 이 류에서 “밀크”란 전유(全乳)나 탈지(脫脂)유[일부 탈지(脫脂)나 완전 탈지(脫脂)를 한 것으로 한정한다]를 말한다.
2. 제0403호에서 요구르트는 농축하거나 향을 첨가할 수 있으며 설탕이나 그 밖의 감미료·과실·견과류·코코아·초콜릿·향신료·커피나 커피 추출물·식물·식물의 부분·곡물이나 베이커리 제품을 함유할 수도 있다. 다만 첨가된 물질이 밀크 성분의 전부나 일부를 대체하기 위한 목적

C:\Users\nguye\AppData\Local\Temp\ipykernel_18652\2355656816.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, 'html.parser')


In [11]:
chapter_df.to_excel("chapter_notes_2024.xlsx",index = False)